<a href="https://colab.research.google.com/github/basse058/deep-learning-challenge/blob/main/Optimizer_Attempt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [28]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
from google.colab import drive
drive.mount('/content/gdrive')
application_df = pd.read_csv('/content/gdrive/MyDrive/charity_data.csv')
application_df.head(5)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_df = application_df.drop(['EIN','NAME','ASK_AMT'], axis=1)
drop_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,1


In [31]:
# Determine the number of unique values in each column.
drop_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
STATUS                     2
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [32]:
# Look at APPLICATION_TYPE value counts for binning
df_counts = drop_df['APPLICATION_TYPE'].value_counts()
df_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(df_counts[df_counts < 400].index)

# Replace in dataframe
for app in application_types_to_replace:
    drop_df['APPLICATION_TYPE'] = drop_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
drop_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning
cl_binning = drop_df['CLASSIFICATION'].value_counts()
cl_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [35]:
# You may find it helpful to look at CLASSIFICATION value counts >1
cl_binning[cl_binning > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [36]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(cl_binning[cl_binning < 400].index)

# Replace in dataframe
for cls in classifications_to_replace:
    drop_df['CLASSIFICATION'] = drop_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
drop_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [37]:
drop_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,1


In [38]:
# Convert categorical data to numeric with `pd.get_dummies`
drop_df = pd.get_dummies(drop_df)
drop_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [39]:
# Split our preprocessed data into our features and target arrays
y = drop_df["IS_SUCCESSFUL"].values
X = drop_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
print(X_train_scaled.shape)
input_features = X_train_scaled.shape[1]

(25724, 43)


Grabbed the following code from 21.2 Activity 5 for the Keras_Tuner.

In [42]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=50,
        max_value=200,
        step=5), activation=activation, input_dim= input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=40,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [43]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [44]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 13s]
val_accuracy: 0.7338775396347046

Best val_accuracy So Far: 0.7349271178245544
Total elapsed time: 00h 32m 53s


In [45]:
# Get best model hyperparameters
top_hyper = tuner.get_best_hyperparameters(1)[0]
top_hyper.values

{'activation': 'relu',
 'first_units': 105,
 'num_layers': 5,
 'units_0': 16,
 'units_1': 26,
 'units_2': 21,
 'units_3': 36,
 'units_4': 1,
 'units_5': 6,
 'units_6': 11,
 'units_7': 16,
 'units_8': 26,
 'units_9': 16,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0014'}

In [46]:
# Compare the performance to the best model
top_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = top_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5541 - accuracy: 0.7349 - 789ms/epoch - 3ms/step
Loss: 0.5541208386421204, Accuracy: 0.7349271178245544


## Compile, Train and Evaluate the Model

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=105, activation="relu", input_dim=input_features))

# Hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn.add(tf.keras.layers.Dense(units=26, activation="relu"))
nn.add(tf.keras.layers.Dense(units=21, activation="relu"))
nn.add(tf.keras.layers.Dense(units=36, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 105)               4620      
                                                                 
 dense_8 (Dense)             (None, 16)                1696      
                                                                 
 dense_9 (Dense)             (None, 26)                442       
                                                                 
 dense_10 (Dense)            (None, 21)                567       
                                                                 
 dense_11 (Dense)            (None, 36)                792       
                                                                 
 dense_12 (Dense)            (None, 1)                 37        
                                                                 
 dense_13 (Dense)            (None, 1)                

In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 6s 3ms/step - loss: 0.6070 - accuracy: 0.7175
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5784 - accuracy: 0.7291
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5678 - accuracy: 0.7289
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5620 - accuracy: 0.7287
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5582 - accuracy: 0.7319
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5561 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5541 - accuracy: 0.7319
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5531 - accuracy: 0.7321
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7322
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7325

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7310 - 673ms/epoch - 3ms/step
Loss: 0.5566616058349609, Accuracy: 0.7309620976448059


In [27]:
# Export our model to HDF5 file
import h5py
model.fit(....)
model.save('model1.h5')

SyntaxError: ignored

In [ ]:
from google.colab import files
files.download("model1.h5")